In [2]:
import numpy as np
import numba
from numba import cuda,types
import matplotlib.pyplot as plt
import pylab
import math

def get_beta(p):
    return p/np.sqrt(1+p**2)
def get_gamma(p):
    return np.sqrt(1+p**2)

numba.cuda.close()

def auto_cuda_config(shape=(32,32), max_threads_per_block=1024):
    threads_x = min(32, shape[1])  
    threads_y = min(32, shape[0])

    blocks_x = (shape[1] + threads_x - 1) // threads_x
    blocks_y = (shape[0] + threads_y - 1) // threads_y

    return (blocks_y, blocks_x), (threads_y, threads_x)

In [5]:
dt = 0.025
steps = int(1/dt)

X = 1000
nx_cell = 10
dx = 1/nx_cell
nx = int(nx_cell*X)
ng_l, ng_r = 0, 0 ### number of guard cells
Nx = nx+ng_l+ng_r
n_cs = 1 ### number of cells in a coarse mesh

Pmax = 3*10**5
Np = int(6000)
dp=2*Pmax/Np
p = np.linspace(Pmax-0.5*dp,0.5*dp-Pmax,Np).astype(np.float64)
# dp = p[0]-p[1]
ptc_gamma = get_gamma(p)
ptc_beta = get_beta(p)
idx_p_0=int(Np/2-1)
idx_n_0=1+idx_p_0

A=10**-18     ### F_rad = A*P**4
B=0.5*10**-5  ## dN/dt = B*P
C=A/(2*B)       ### P_e = C*P**3
Fcen=0

n_e_ini = 6
n_p_ini = n_e_ini-4
idx_ini=idx_p_0-1
IC_e, IC_p = np.zeros((Np,Nx)).astype(np.float64) , np.zeros((Np,Nx)).astype(np.float64)
IC_e[idx_ini][ng_l::] = n_e_ini*np.ones(Nx-ng_l)
IC_p[idx_p_0][ng_l::] = (0.5*n_p_ini)*np.ones(Nx-ng_l)
IC_p[idx_n_0][ng_l::] = (0.5*n_p_ini)*np.ones(Nx-ng_l)
j0=-n_e_ini*get_beta(p[idx_ini])

E_ini=-np.zeros(Nx+1)
E_max=10000


idx_cr=2499
crit_eng=C*p[idx_cr]**3

mapping=[]
for i in range(1200):
    mapping.append(i)
    mapping.append(int(X*p[idx_cr]**3/p[i]**3//1))
    mapping.append(1-X*p[idx_cr]**3/p[i]**3%1)
    mapping.append(int(X*p[idx_cr]**3/p[i]**3//1+1))
    mapping.append(X*p[idx_cr]**3/p[i]**3%1)
    mapping.append(int(idx_p_0-(C*p[i]**3-50)//100))
    mapping.append((1-(C*p[i]**3-50)%100/100)*B*p[i])
    mapping.append(int(idx_p_0-(C*p[i]**3-50)//100-1))
    mapping.append(((C*p[i]**3-50)%100/100)*B*p[i])
    mapping.append(Np-1-i)
    mapping.append(int(X*p[idx_cr]**3/p[i]**3//1))
    mapping.append(1-X*p[idx_cr]**3/p[i]**3%1)
    mapping.append(int(X*p[idx_cr]**3/p[i]**3//1+1))
    mapping.append(X*p[idx_cr]**3/p[i]**3%1)
    mapping.append(int(idx_n_0+abs(C*p[i]**3+50)//100-1))
    mapping.append((1-(C*p[i]**3-50)%100/100)*B*p[i])
    mapping.append(int(idx_n_0+abs(C*p[i]**3+50)//100))
    mapping.append(((C*p[i]**3-50)%100/100)*B*p[i])
tem_ph=np.zeros((X-1)*(mapping[16]-mapping[7]+1)*X)
xpd_ph_size=(mapping[16]-mapping[7]+1)*X
blocks_per_grid, threads_per_block = auto_cuda_config((X-1,(mapping[16]-mapping[7]+1)*X))
crit_eng

12.5375375125

In [6]:
@cuda.jit
def copy_var(a,b):
    i_start = cuda.grid(1) 
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    for i in range(i_start, b.size, threads_per_grid):  
        b[i] = a[i]

@cuda.jit
def copy_var2(a,b):
    i,j = cuda.grid(2) 
    if i <a.shape[0] and j< a.shape[1]:
        b[i,j]=a[i,j]
        
@cuda.jit
def add_var(a,b,c):
    i_start = cuda.grid(1) 
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    for i in range(i_start, c.size, threads_per_grid):  
        c[i] = a[i]+b[i]

@cuda.jit
def clear_var(a):
    i_start = cuda.grid(1) 
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    for i in range(i_start, a.size, threads_per_grid):  
        a[i] = 0

@cuda.jit
def update_E(J,E):
    i_start = cuda.grid(1)
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    for i in range(i_start, E.size, threads_per_grid):
            E[i]-=J[i]*dt
        

@cuda.jit
def operate_P(up_b,low_b,xpd1,xpd2,v,dxpd):
    i_start = cuda.grid(1)
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    for i in range(i_start, dxpd.size, threads_per_grid):
        if i > (up_b[0]-50)*Nx and i<(low_b[0]+50)*Nx:
            idy = i//Nx
            idx = i%Nx
            cell=0.5*(xpd1[i]+xpd2[i])
            cellm1=0.5*(xpd1[i-1]+xpd2[i-1])
            cellp1=0.5*(xpd1[i+1]+xpd2[i+1])
            if idy <= idx_p_0:
                if idx == 0:
                    dxpd[i] += -cell*v[idy]*dt/dx
                if idx > 0:
                    dxpd[i] += (cellm1-cell)*v[idy]*dt/dx
            if idy >= idx_n_0:
                if idx == Nx-1:
                    dxpd[i] += cell*v[idy]*dt/dx
                if idx < Nx-1:
                    dxpd[i] += (cell-cellp1)*v[idy]*dt/dx

@cuda.jit
def get_J_gpu(xpd_e1,xpd_e2,xpd_p1,xpd_p2,v,J):
    i_start = cuda.grid(1) 
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x 
    for i in range(i_start, J.size, threads_per_grid):  
        J[i]=-j0
        if i> ng_l and i <Nx:
            for k in range(Np):
                if k <= idx_p_0:
                    J[i] += 0.5*(xpd_p1[k*Nx+i-1]+xpd_p2[k*Nx+i-1]-xpd_e1[k*Nx+i-1]-xpd_e2[k*Nx+i-1])*v[k]
                if k >= idx_n_0:
                    J[i] += 0.5*(xpd_p1[k*Nx+i]+xpd_p2[k*Nx+i]-xpd_e1[k*Nx+i]-xpd_e2[k*Nx+i-1])*v[k]
        if i ==Nx:
            for k in range(Np):
                J[i] += 0.5*(xpd_p1[k*Nx+i-1]+xpd_p2[k*Nx+i-1]-xpd_e1[k*Nx+i-1]-xpd_e2[k*Nx+i-1])*v[k]
        if i <= ng_l:## or i >=Nx-ng_r:
            J[i]=0
        # if abs(J[i]) < 0.005*abs(j0):
        #     J[i]=0
        

                            
@cuda.jit
def operate_E(up_b,low_b,xpd1,xpd2,v,p,E,spec,dxpd):
    i_start = cuda.grid(1)
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    for i in range(i_start, dxpd.size, threads_per_grid):
        if i > (up_b[0]-50)*Nx and i<(low_b[0]+50)*Nx:
            idy = i//Nx
            idx = i%Nx
            dxpd[i]=0
            if idx<ng_l:
                dpdt_U=0
                dpdt_M=0
                dpdt_D=0
            if idx==ng_l:
                if idy < idx_p_0:
                    dpdt_1_U = spec*(E[idx+1])-A*p[idy-1]**4+Fcen
                    dpdt_2_U = spec*(E[idx+1])-A*(p[idy-1]+dpdt_1_U*dt/2)**4+Fcen
                    dpdt_3_U = spec*(E[idx+1])-A*(p[idy-1]+dpdt_2_U*dt/2)**4+Fcen
                    dpdt_4_U = spec*(E[idx+1])-A*(p[idy-1]+dpdt_3_U*dt)**4+Fcen
                    dpdt_U = dpdt_1_U/6+dpdt_2_U/3+dpdt_3_U/3+dpdt_4_U/6
                    dpdt_1_M = spec*(E[idx+1])-A*p[idy]**4+Fcen
                    dpdt_2_M = spec*(E[idx+1])-A*(p[idy]+dpdt_1_M*dt/2)**4+Fcen
                    dpdt_3_M = spec*(E[idx+1])-A*(p[idy]+dpdt_2_M*dt/2)**4+Fcen
                    dpdt_4_M = spec*(E[idx+1])-A*(p[idy]+dpdt_3_M*dt)**4+Fcen
                    dpdt_M = dpdt_1_M/6+dpdt_2_M/3+dpdt_3_M/3+dpdt_4_M/6
                    dpdt_1_D = spec*(E[idx+1])-A*p[idy+1]**4+Fcen
                    dpdt_2_D = spec*(E[idx+1])-A*(p[idy+1]+dpdt_1_D*dt/2)**4+Fcen
                    dpdt_3_D = spec*(E[idx+1])-A*(p[idy+1]+dpdt_2_D*dt/2)**4+Fcen
                    dpdt_4_D = spec*(E[idx+1])-A*(p[idy+1]+dpdt_3_D*dt)**4+Fcen
                    dpdt_D = dpdt_1_D/6+dpdt_2_D/3+dpdt_3_D/3+dpdt_4_D/6
                if idy == idx_p_0:
                    dpdt_1_U = spec*(E[idx+1])-A*p[idy-1]**4+Fcen
                    dpdt_2_U = spec*(E[idx+1])-A*(p[idy-1]+dpdt_1_U*dt/2)**4+Fcen
                    dpdt_3_U = spec*(E[idx+1])-A*(p[idy-1]+dpdt_2_U*dt/2)**4+Fcen
                    dpdt_4_U = spec*(E[idx+1])-A*(p[idy-1]+dpdt_3_U*dt)**4+Fcen
                    dpdt_U = dpdt_1_U/6+dpdt_2_U/3+dpdt_3_U/3+dpdt_4_U/6
                    dpdt_1_M = spec*(E[idx+1])-A*p[idy]**4+Fcen
                    dpdt_2_M = spec*(E[idx+1])-A*(p[idy]+dpdt_1_M*dt/2)**4+Fcen
                    dpdt_3_M = spec*(E[idx+1])-A*(p[idy]+dpdt_2_M*dt/2)**4+Fcen
                    dpdt_4_M = spec*(E[idx+1])-A*(p[idy]+dpdt_3_M*dt)**4+Fcen
                    dpdt_M = dpdt_1_M/6+dpdt_2_M/3+dpdt_3_M/3+dpdt_4_M/6
                    dpdt_1_D = spec*(E[idx+1])+A*p[idy+1]**4+Fcen
                    dpdt_2_D = spec*(E[idx+1])+A*(p[idy+1]+dpdt_1_D*dt/2)**4+Fcen
                    dpdt_3_D = spec*(E[idx+1])+A*(p[idy+1]+dpdt_2_D*dt/2)**4+Fcen
                    dpdt_4_D = spec*(E[idx+1])+A*(p[idy+1]+dpdt_3_D*dt)**4+Fcen
                    dpdt_D = dpdt_1_D/6+dpdt_2_D/3+dpdt_3_D/3+dpdt_4_D/6 
                if idy == idx_n_0:
                    dpdt_1_U = spec*(E[idx+1])-A*p[idy-1]**4+Fcen
                    dpdt_2_U = spec*(E[idx+1])-A*(p[idy-1]+dpdt_1_U*dt/2)**4+Fcen
                    dpdt_3_U = spec*(E[idx+1])-A*(p[idy-1]+dpdt_2_U*dt/2)**4+Fcen
                    dpdt_4_U = spec*(E[idx+1])-A*(p[idy-1]+dpdt_3_U*dt)**4+Fcen
                    dpdt_U = dpdt_1_U/6+dpdt_2_U/3+dpdt_3_U/3+dpdt_4_U/6
                    dpdt_1_M = spec*(E[idx+1])+A*p[idy]**4+Fcen
                    dpdt_2_M = spec*(E[idx+1])+A*(p[idy]+dpdt_1_M*dt/2)**4+Fcen
                    dpdt_3_M = spec*(E[idx+1])+A*(p[idy]+dpdt_2_M*dt/2)**4+Fcen
                    dpdt_4_M = spec*(E[idx+1])+A*(p[idy]+dpdt_3_M*dt)**4+Fcen
                    dpdt_M = dpdt_1_M/6+dpdt_2_M/3+dpdt_3_M/3+dpdt_4_M/6
                    dpdt_1_D = spec*(E[idx+1])+A*p[idy+1]**4+Fcen
                    dpdt_2_D = spec*(E[idx+1])+A*(p[idy+1]+dpdt_1_D*dt/2)**4+Fcen
                    dpdt_3_D = spec*(E[idx+1])+A*(p[idy+1]+dpdt_2_D*dt/2)**4+Fcen
                    dpdt_4_D = spec*(E[idx+1])+A*(p[idy+1]+dpdt_3_D*dt)**4+Fcen
                    dpdt_D = dpdt_1_D/6+dpdt_2_D/3+dpdt_3_D/3+dpdt_4_D/6 
                if idy > idx_n_0:
                    dpdt_1_U = spec*(E[idx+1])+A*p[idy-1]**4+Fcen
                    dpdt_2_U = spec*(E[idx+1])+A*(p[idy-1]+dpdt_1_U*dt/2)**4+Fcen
                    dpdt_3_U = spec*(E[idx+1])+A*(p[idy-1]+dpdt_2_U*dt/2)**4+Fcen
                    dpdt_4_U = spec*(E[idx+1])+A*(p[idy-1]+dpdt_3_U*dt)**4+Fcen
                    dpdt_U = dpdt_1_U/6+dpdt_2_U/3+dpdt_3_U/3+dpdt_4_U/6
                    dpdt_1_M = spec*(E[idx+1])+A*p[idy]**4+Fcen
                    dpdt_2_M = spec*(E[idx+1])+A*(p[idy]+dpdt_1_M*dt/2)**4+Fcen
                    dpdt_3_M = spec*(E[idx+1])+A*(p[idy]+dpdt_2_M*dt/2)**4+Fcen
                    dpdt_4_M = spec*(E[idx+1])+A*(p[idy]+dpdt_3_M*dt)**4+Fcen
                    dpdt_M = dpdt_1_M/6+dpdt_2_M/3+dpdt_3_M/3+dpdt_4_M/6
                    dpdt_1_D = spec*(E[idx+1])+A*p[idy+1]**4+Fcen
                    dpdt_2_D = spec*(E[idx+1])+A*(p[idy+1]+dpdt_1_D*dt/2)**4+Fcen
                    dpdt_3_D = spec*(E[idx+1])+A*(p[idy+1]+dpdt_2_D*dt/2)**4+Fcen
                    dpdt_4_D = spec*(E[idx+1])+A*(p[idy+1]+dpdt_3_D*dt)**4+Fcen
                    dpdt_D = dpdt_1_D/6+dpdt_2_D/3+dpdt_3_D/3+dpdt_4_D/6
            if idx>ng_l:
                if idy < idx_p_0:
                    dpdt_1_U = spec*(E[idx]+E[idx+1])*0.5-A*p[idy-1]**4+Fcen
                    dpdt_2_U = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy-1]+dpdt_1_U*dt/2)**4+Fcen
                    dpdt_3_U = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy-1]+dpdt_2_U*dt/2)**4+Fcen
                    dpdt_4_U = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy-1]+dpdt_3_U*dt)**4+Fcen
                    dpdt_U = dpdt_1_U/6+dpdt_2_U/3+dpdt_3_U/3+dpdt_4_U/6
                    dpdt_1_M = spec*(E[idx]+E[idx+1])*0.5-A*p[idy]**4+Fcen
                    dpdt_2_M = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy]+dpdt_1_M*dt/2)**4+Fcen
                    dpdt_3_M = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy]+dpdt_2_M*dt/2)**4+Fcen
                    dpdt_4_M = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy]+dpdt_3_M*dt)**4+Fcen
                    dpdt_M = dpdt_1_M/6+dpdt_2_M/3+dpdt_3_M/3+dpdt_4_M/6
                    dpdt_1_D = spec*(E[idx]+E[idx+1])*0.5-A*p[idy+1]**4+Fcen
                    dpdt_2_D = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy+1]+dpdt_1_D*dt/2)**4+Fcen
                    dpdt_3_D = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy+1]+dpdt_2_D*dt/2)**4+Fcen
                    dpdt_4_D = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy+1]+dpdt_3_D*dt)**4+Fcen
                    dpdt_D = dpdt_1_D/6+dpdt_2_D/3+dpdt_3_D/3+dpdt_4_D/6
                if idy == idx_p_0:
                    dpdt_1_U = spec*(E[idx]+E[idx+1])*0.5-A*p[idy-1]**4+Fcen
                    dpdt_2_U = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy-1]+dpdt_1_U*dt/2)**4+Fcen
                    dpdt_3_U = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy-1]+dpdt_2_U*dt/2)**4+Fcen
                    dpdt_4_U = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy-1]+dpdt_3_U*dt)**4+Fcen
                    dpdt_U = dpdt_1_U/6+dpdt_2_U/3+dpdt_3_U/3+dpdt_4_U/6
                    dpdt_1_M = spec*(E[idx]+E[idx+1])*0.5-A*p[idy]**4+Fcen
                    dpdt_2_M = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy]+dpdt_1_M*dt/2)**4+Fcen
                    dpdt_3_M = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy]+dpdt_2_M*dt/2)**4+Fcen
                    dpdt_4_M = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy]+dpdt_3_M*dt)**4+Fcen
                    dpdt_M = dpdt_1_M/6+dpdt_2_M/3+dpdt_3_M/3+dpdt_4_M/6
                    dpdt_1_D = spec*(E[idx]+E[idx+1])*0.5+A*p[idy+1]**4+Fcen
                    dpdt_2_D = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy+1]+dpdt_1_D*dt/2)**4+Fcen
                    dpdt_3_D = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy+1]+dpdt_2_D*dt/2)**4+Fcen
                    dpdt_4_D = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy+1]+dpdt_3_D*dt)**4+Fcen
                    dpdt_D = dpdt_1_D/6+dpdt_2_D/3+dpdt_3_D/3+dpdt_4_D/6
                if idy == idx_n_0:
                    dpdt_1_U = spec*(E[idx]+E[idx+1])*0.5-A*p[idy-1]**4+Fcen
                    dpdt_2_U = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy-1]+dpdt_1_U*dt/2)**4+Fcen
                    dpdt_3_U = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy-1]+dpdt_2_U*dt/2)**4+Fcen
                    dpdt_4_U = spec*(E[idx]+E[idx+1])*0.5-A*(p[idy-1]+dpdt_3_U*dt)**4+Fcen
                    dpdt_U = dpdt_1_U/6+dpdt_2_U/3+dpdt_3_U/3+dpdt_4_U/6
                    dpdt_1_M = spec*(E[idx]+E[idx+1])*0.5+A*p[idy]**4+Fcen
                    dpdt_2_M = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy]+dpdt_1_M*dt/2)**4+Fcen
                    dpdt_3_M = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy]+dpdt_2_M*dt/2)**4+Fcen
                    dpdt_4_M = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy]+dpdt_3_M*dt)**4+Fcen
                    dpdt_M = dpdt_1_M/6+dpdt_2_M/3+dpdt_3_M/3+dpdt_4_M/6
                    dpdt_1_D = spec*(E[idx]+E[idx+1])*0.5+A*p[idy+1]**4+Fcen
                    dpdt_2_D = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy+1]+dpdt_1_D*dt/2)**4+Fcen
                    dpdt_3_D = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy+1]+dpdt_2_D*dt/2)**4+Fcen
                    dpdt_4_D = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy+1]+dpdt_3_D*dt)**4+Fcen
                    dpdt_D = dpdt_1_D/6+dpdt_2_D/3+dpdt_3_D/3+dpdt_4_D/6
                if idy > idx_n_0:
                    dpdt_1_U = spec*(E[idx]+E[idx+1])*0.5+A*p[idy-1]**4+Fcen
                    dpdt_2_U = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy-1]+dpdt_1_U*dt/2)**4+Fcen
                    dpdt_3_U = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy-1]+dpdt_2_U*dt/2)**4+Fcen
                    dpdt_4_U = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy-1]+dpdt_3_U*dt)**4+Fcen
                    dpdt_U = dpdt_1_U/6+dpdt_2_U/3+dpdt_3_U/3+dpdt_4_U/6
                    dpdt_1_M = spec*(E[idx]+E[idx+1])*0.5+A*p[idy]**4+Fcen
                    dpdt_2_M = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy]+dpdt_1_M*dt/2)**4+Fcen
                    dpdt_3_M = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy]+dpdt_2_M*dt/2)**4+Fcen
                    dpdt_4_M = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy]+dpdt_3_M*dt)**4+Fcen
                    dpdt_M = dpdt_1_M/6+dpdt_2_M/3+dpdt_3_M/3+dpdt_4_M/6
                    dpdt_1_D = spec*(E[idx]+E[idx+1])*0.5+A*p[idy+1]**4+Fcen
                    dpdt_2_D = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy+1]+dpdt_1_D*dt/2)**4+Fcen
                    dpdt_3_D = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy+1]+dpdt_2_D*dt/2)**4+Fcen
                    dpdt_4_D = spec*(E[idx]+E[idx+1])*0.5+A*(p[idy+1]+dpdt_3_D*dt)**4+Fcen
                    dpdt_D = dpdt_1_D/6+dpdt_2_D/3+dpdt_3_D/3+dpdt_4_D/6
            if v[idy]>=0:
                if dpdt_M <= 0:
                    dxpd[i]-=0.5*(xpd1[i-Nx]+xpd2[i-Nx])*dpdt_U*dt/dp
                    dxpd[i]+=0.5*(xpd1[i]+xpd2[i])*dpdt_M*dt/dp
                    if dpdt_D>0:
                        dxpd[i]+=0.5*(xpd1[i+Nx]+xpd2[i+Nx])*dpdt_D*dt/dp
                if dpdt_M > 0:
                    dxpd[i]+=0.5*(xpd1[i+Nx]+xpd2[i+Nx])*dpdt_D*dt/dp
                    dxpd[i]-=0.5*(xpd1[i]+xpd2[i])*dpdt_M*dt/dp
                    if dpdt_U < 0:
                        dxpd[i]-= 0.5*(xpd1[i-Nx]+xpd2[i-Nx])*dpdt_U*dt/dp
            if v[idy]<0:
                if dpdt_M >= 0:
                    dxpd[i]+=0.5*(xpd1[i+Nx]+xpd2[i+Nx])*dpdt_D*dt/dp
                    dxpd[i]-=0.5*(xpd1[i]+xpd2[i])*dpdt_M*dt/dp
                    if dpdt_U<0:
                        dxpd[i]-=0.5*(xpd1[i-Nx]+xpd2[i-Nx])*dpdt_U*dt/dp
                if dpdt_M < 0:
                    dxpd[i]+=0.5*(xpd1[i]+xpd2[i])*dpdt_M*dt/dp
                    dxpd[i]-=0.5*(xpd1[i-Nx]+xpd2[i-Nx])*dpdt_U*dt/dp
                    if dpdt_D >0:
                        dxpd[i]+=0.5*(xpd1[i+Nx]+xpd2[i+Nx])*dpdt_D*dt/dp

@cuda.jit
def ext_ptc_1(E,max,dxpd):
    i_start = cuda.grid(1)
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    for i in range(i_start, dxpd.size, threads_per_grid):
        dxpd[i]=0
        if i ==(idx_p_0)*Nx+ng_l:
            dxpd[i]+=E[ng_l+1]/E_max*(abs(E[ng_l+1])/E_max)*dt/dx
            if E[ng_l+1]*(abs(E[ng_l+1])/E_max)*dt/dx>max*dt/dx:
                dxpd[i]=max*dt/dx
            if E[ng_l+1]*(abs(E[ng_l+1])/E_max)*dt/dx<-max*dt/dx:
                dxpd[i]=-max*dt/dx
            # dxpd[i]=(E[ng_l+1]-E[ng_l])/dx

@cuda.jit
def ext_ptc_2(xpd_e,xpd_p,dxpd):
    i_start = cuda.grid(1)
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    for i in range(i_start, dxpd.size, threads_per_grid):
        if dxpd[i]>0:
            xpd_p[i]+=dxpd[i]
        if dxpd[i]<0:
            xpd_e[i]-=dxpd[i]            

@cuda.jit
def get_box(xpd,box_xpd):
    i_start = cuda.grid(1)
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    for i in range(i_start, box_xpd.size, threads_per_grid):
        idy = i // nx
        idx = i % nx
        box_xpd[i] =xpd[idx+ng_l+Nx*idy]

@cuda.jit
def bound_check(up_b,low_b,c,thr=10**-20):
    shared_up_b = cuda.shared.array(1, dtype=types.int32)
    shared_low_b = cuda.shared.array(1, dtype=types.int32)
    tid = cuda.threadIdx.x
    i_start = cuda.grid(1)
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    if tid == 0:
        shared_up_b[0] = Np
        shared_low_b[0] = 0
    cuda.syncthreads()    
    for i in range(i_start, c.size, threads_per_grid):
        idy = i // X  
        if c[i] > thr:
            cuda.atomic.min(shared_up_b, 0, idy)
            cuda.atomic.max(shared_low_b, 0, idy)
    cuda.syncthreads()  
    if tid == 0:
        cuda.atomic.min(up_b, 0, shared_up_b[0])
        cuda.atomic.max(low_b, 0, shared_low_b[0])

@cuda.jit
def update_Tem_ph_1(Tem_ph, Tem_ph2):
    i_start = cuda.grid(1)
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    for i in range(i_start, Tem_ph.size, threads_per_grid):
        Tem_ph[i]=0
        if i<Tem_ph.size-xpd_ph_size:
            Tem_ph[i] = Tem_ph2[i+xpd_ph_size]  


@cuda.jit
def update_Tem_ph_2(up_e,low_e,up_p,low_p,Tem_ph,xpd_e,xpd_p,mapping):
    up_b=min(up_e[0],up_p[0])
    low_b=max(low_e[0],low_p[0])
    start_index=mapping[7]
    i_start = cuda.grid(1)
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    for i in range(i_start, Tem_ph.size, threads_per_grid):
        Tid = i // xpd_ph_size
        idy = (i % xpd_ph_size) // X + mapping[7]
        idx = (i % xpd_ph_size) % X
        for m in range(int(mapping.size/18)):
            if mapping[18*m]>=up_b-10 and mapping[18*m]<=idx_cr:
                p_idx=int(mapping[18*m]*X+idx-Tid)
                if mapping[18*m+5] == idy and mapping[18*m+1] == Tid:
                    if idx-Tid>=0:
                        cuda.atomic.add(Tem_ph, i, (xpd_e[p_idx] + xpd_p[p_idx]) * mapping[18*m+6] * dt * mapping[18*m+2])
                if mapping[18*m+5] == idy and mapping[18*m+3] == Tid:
                    if idx-Tid>=0:
                        cuda.atomic.add(Tem_ph, i, (xpd_e[p_idx] + xpd_p[p_idx]) * mapping[18*m+6] * dt * mapping[18*m+4])
                if mapping[18*m+7] == idy and mapping[18*m+1] == Tid:
                    if idx-Tid>=0:
                        cuda.atomic.add(Tem_ph, i, (xpd_e[p_idx] + xpd_p[p_idx]) * mapping[18*m+8] * dt * mapping[18*m+2])
                if mapping[18*m+7] == idy and mapping[18*m+3] == Tid:
                    if idx-Tid>=0:
                        cuda.atomic.add(Tem_ph, i, (xpd_e[p_idx] + xpd_p[p_idx]) * mapping[18*m+8] * dt * mapping[18*m+4])
            if mapping[18*m+9]<=low_b+10 and mapping[18*m+9]>=Np-1-idx_cr:
                p_idx=int(mapping[18*m+9]*X+idx+Tid)
                if mapping[18*m+14] == idy and mapping[18*m+10] == Tid:
                    if idx+Tid<X:
                        cuda.atomic.add(Tem_ph, i, (xpd_e[p_idx] + xpd_p[p_idx]) * mapping[18*m+15] * dt * mapping[18*m+11])
                if mapping[18*m+14] == idy and mapping[18*m+12] == Tid:
                    if idx+Tid<X:
                        cuda.atomic.add(Tem_ph, i, (xpd_e[p_idx] + xpd_p[p_idx]) * mapping[18*m+15] * dt * mapping[18*m+13])
                if mapping[18*m+16] == idy and mapping[18*m+10] == Tid:
                    if idx+Tid<X:
                        cuda.atomic.add(Tem_ph, i, (xpd_e[p_idx] + xpd_p[p_idx]) * mapping[18*m+17] * dt * mapping[18*m+11])
                if mapping[18*m+16] == idy and mapping[18*m+12] == Tid:
                    if idx+Tid<X:
                        cuda.atomic.add(Tem_ph, i, (xpd_e[p_idx] + xpd_p[p_idx]) * mapping[18*m+17] * dt * mapping[18*m+13])




@cuda.jit
def inject_pairs(Tem_ph,xpd_e,xpd_p,mapping):
    i_start = cuda.grid(1)
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    for i in range(i_start, xpd_e.size, threads_per_grid):
        idx=i % nx
        idy=i // nx
        idX=idx// nx_cell
        if mapping[7]<=idy and idy<=mapping[16]:
            cuda.atomic.add(xpd_e,i,Tem_ph[int((idy-mapping[7])*X+idX)])
            cuda.atomic.add(xpd_p,i,Tem_ph[int((idy-mapping[7])*X+idX)])

@cuda.jit
def get_coarse_box(xpd,coarse_xpd):
    i_start = cuda.grid(1)
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    for i in range(i_start, coarse_xpd.size, threads_per_grid):
        coarse_xpd[i]=0
        for j in range(nx_cell):
            cuda.atomic.add(coarse_xpd,i,xpd[i*nx_cell+j]*dx)

@cuda.jit
def get_box(xpd,box_xpd):
    i_start = cuda.grid(1)
    threads_per_grid = cuda.blockDim.x * cuda.gridDim.x
    for i in range(i_start, box_xpd.size, threads_per_grid):
        idy = i // nx
        idx = i % nx
        box_xpd[i] =xpd[idx+ng_l+Nx*idy]

In [7]:
data_adr='./Data/RS_x1000/'
tot_steps=steps*10000
dev_beta = cuda.to_device(ptc_beta)
dev_p=cuda.to_device(p)
dev_E = cuda.to_device(E_ini)
dev_dxpd = cuda.to_device(np.zeros(IC_e.flatten().size))
dev_dxpd2 = cuda.to_device(np.zeros(IC_e.flatten().size))
dev_xpd1_e = cuda.to_device(IC_e.flatten())
dev_xpd2_e = cuda.to_device(IC_e.flatten()) 
dev_xpd1_p = cuda.to_device(IC_p.flatten())
dev_xpd2_p = cuda.to_device(IC_p.flatten())
dev_J = cuda.to_device(np.zeros(E_ini.size)) 
dev_coarse_xpd_e=cuda.to_device(np.zeros(X*Np))
dev_coarse_xpd_p=cuda.to_device(np.zeros(X*Np))
dev_low_e=cuda.to_device(np.array([0]))
dev_up_e=cuda.to_device(np.array([Np]))
dev_low_p=cuda.to_device(np.array([0]))
dev_up_p=cuda.to_device(np.array([Np]))
dev_mapping=cuda.to_device(mapping)
dev_eng=cuda.to_device(np.zeros(1))
dev_Tem_ph=cuda.to_device(tem_ph)
dev_Tem_ph2=cuda.to_device(tem_ph)
get_coarse_box[1024,1024](dev_xpd1_e,dev_coarse_xpd_e)
cuda.synchronize()
get_coarse_box[1024,1024](dev_xpd1_p,dev_coarse_xpd_p)
cuda.synchronize()

In [ ]:
for i in range(tot_steps):
    if i % (steps) ==0:
        copy_var[1024,1024](dev_Tem_ph, dev_Tem_ph2)
        cuda.synchronize()
        update_Tem_ph_1[1024,1024](dev_Tem_ph, dev_Tem_ph2)
        cuda.synchronize()
        inject_pairs[1024,1024](dev_Tem_ph,dev_xpd1_e,dev_xpd1_p,dev_mapping)
        cuda.synchronize()
        # add_var[1024,1024](dev_dxpd,dev_xpd1_e,dev_xpd1_e)
        # cuda.synchronize()
        # add_var[1024,1024](dev_dxpd,dev_xpd1_p,dev_xpd1_p)
        # cuda.synchronize()
        E=dev_E.copy_to_host()
        np.save(data_adr+'E' + '0'*(5-len(str(int(i//steps))))+str(int(i//steps)),E)
        xpd_e=np.reshape(dev_coarse_xpd_e.copy_to_host(),(Np,X))
        xpd_p=np.reshape(dev_coarse_xpd_p.copy_to_host(),(Np,X))
        J=dev_J.copy_to_host()
        np.save(data_adr+'e_xpd' + '0'*(5-len(str(int(i//steps))))+str(int(i//steps)),xpd_e)
        np.save(data_adr+'p_xpd' + '0'*(5-len(str(int(i//steps))))+str(int(i//steps)),xpd_p)
        np.save(data_adr+'J' + '0'*(5-len(str(int(i//steps))))+str(int(i//steps)),J)

    bound_check[1024,1024](dev_up_e,dev_low_e,dev_coarse_xpd_e,0)
    cuda.synchronize()
    bound_check[1024,1024](dev_up_p,dev_low_p,dev_coarse_xpd_p,0)
    cuda.synchronize()

    for j in range(10):
        operate_E[1024,1024](dev_up_e,dev_low_e,dev_xpd1_e,dev_xpd2_e,dev_beta,dev_p,dev_E,-1,dev_dxpd)
        cuda.synchronize()
        operate_P[1024,1024](dev_up_e,dev_low_e,dev_xpd1_e,dev_xpd2_e,dev_beta,dev_dxpd)
        cuda.synchronize()
        add_var[1024,1024](dev_xpd1_e,dev_dxpd,dev_xpd2_e)
        cuda.synchronize()
    
    for k in range(10):
        operate_E[1024,1024](dev_up_p,dev_low_p,dev_xpd1_p,dev_xpd2_p,dev_beta,dev_p,dev_E,1,dev_dxpd)
        cuda.synchronize()
        operate_P[1024,1024](dev_up_p,dev_low_p,dev_xpd1_p,dev_xpd2_p,dev_beta,dev_dxpd)
        cuda.synchronize()
        add_var[1024,1024](dev_xpd1_p,dev_dxpd,dev_xpd2_p)
        cuda.synchronize()

    get_J_gpu[1024,1024](dev_xpd1_e,dev_xpd2_e,dev_xpd1_p,dev_xpd2_p,dev_beta,dev_J)
    cuda.synchronize()
    
    copy_var[1024,1024](dev_xpd2_e,dev_xpd1_e)
    cuda.synchronize()
    copy_var[1024,1024](dev_xpd2_p,dev_xpd1_p)
    cuda.synchronize()
    
    
    update_E[1024,1024](dev_J,dev_E)
    cuda.synchronize()
    clear_var[1024,1024](dev_dxpd)
    cuda.synchronize()

    # ext_ptc_1[1024,1024](dev_E,4,dev_dxpd)
    # cuda.synchronize()
    # ext_ptc_2[1024,1024](dev_xpd1_e,dev_xpd1_p, dev_dxpd)
    # cuda.synchronize()
    # # get_eng_boundary[1024,1024](dev_dxpd,dev_dxpd,dev_p,dev_eng_inj,0)
    # # cuda.synchronize()
    # clear_var[1024,1024](dev_dxpd)
    # cuda.synchronize()

    get_coarse_box[1024,1024](dev_xpd1_e,dev_coarse_xpd_e)
    cuda.synchronize()
    get_coarse_box[1024,1024](dev_xpd1_p,dev_coarse_xpd_p)
    cuda.synchronize()
    update_Tem_ph_2[1024,1024](dev_up_e,dev_low_e,dev_up_p,dev_low_p,dev_Tem_ph,dev_coarse_xpd_e,dev_coarse_xpd_p,dev_mapping)
    cuda.synchronize()